In [1]:
WORKSPACE_PATH = 'workspace'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2

Lade JSON

In [3]:
import json

def json_read(data_path):
    with open(data_path, 'r') as data:
        data = json.load(data)
    return data

In [4]:
JSON_NAME = '/instances_default.json'
DATA_PATH = ANNOTATION_PATH+'/original'+ JSON_NAME
data = json_read(DATA_PATH)

Bennene Bilder im Verzeichnis images und in der Annotationsdatei um

In [5]:
import os

def rename_images(IMAGE_PATH):
    index=1
    img  = os.listdir(IMAGE_PATH)
    for i in img:
        new_name = f"{index:012d}"+'.jpeg'
        old_path = os.path.join(IMAGE_PATH, i)
        new_path = os.path.join(IMAGE_PATH, new_name)
        os.rename(old_path, new_path)
        index += 1

def rename_images_in_annotation():
    index=1
    for img in data['images']:
        new_name = f"{index:012d}"+'.jpeg'
        img['file_name'] = new_name 
        index += 1

rename_images(IMAGE_PATH+'/original')
rename_images_in_annotation()

with open(ANNOTATION_PATH+'/original' + JSON_NAME, 'w') as destination_file:
    json.dump(data, destination_file, indent=2)  

In [6]:
images = []
bounding_boxes = []
labels = []
image_height = 1366
image_width = 768

for image in data['images']:
    one_image = []
    one_image.append(image['id'])
    one_image.append(image['file_name'])
    images.append(one_image)

for annotation in data['annotations']:
    one_box = []
    box_and_id = []
    one_box.append(annotation['bbox'])
    bounding_boxes.append(one_box)

Resize

In [7]:
def resize_imgaes(input_path, output_path, height, width):
    image =  cv2.imread(input_path)
    resized_image = cv2.resize(image, (height, width))
    cv2.imwrite(output_path, resized_image)    

def resize_bounding_boxes_and_area(input_path,bounding_boxes, height, width):
    image =  cv2.imread(input_path)
    height_ratio = height / image.shape[0]
    width_ratio = width / image.shape[1]
    
    for box in bounding_boxes:
        resized_boxes = []
        resized_area = 0

        x = np.round(box[0]*width_ratio,2)
        y = np.round(box[1]*height_ratio,2)
        x_width = np.round(box[2]*width_ratio,2)
        y_height = np.round(box[3]*height_ratio,2)
        resized_boxes.append([x, y, x_width, y_height])

        resized_area = x_width * y_height

    return x, y, x_width, y_height, resized_area


In [8]:
import os

new_height = 224
new_width = 224
OUTPUT_PATH_IMAGES = IMAGE_PATH + '/' + str(new_height) + 'x' + str(new_width)
OUTPUTPATH_ANNOTATIONS = ANNOTATION_PATH + '/' + str(new_height) + 'x' + str(new_width)

os.makedirs(OUTPUT_PATH_IMAGES, exist_ok=True)
os.makedirs(OUTPUTPATH_ANNOTATIONS, exist_ok=True)

#Copy JSON Doc in new ANNOTATIONPATH
with open(DATA_PATH, 'r') as source_file:
    data = json.load(source_file)
with open(OUTPUTPATH_ANNOTATIONS + JSON_NAME, 'w') as destination_file:
    json.dump(data, destination_file, indent=2)    


In [9]:
#Resize Images
for img in range(len(images)):
    resize_imgaes(IMAGE_PATH+'/original'+ '/' +images[img][1], OUTPUT_PATH_IMAGES+ '/' +images[img][1], new_height, new_width)


with open(OUTPUTPATH_ANNOTATIONS + JSON_NAME, 'r') as json_data:
    data = json.load(json_data)


#Resize BBox and area
for i, annotation in enumerate(data['annotations']):
    calculate_resize = resize_bounding_boxes_and_area(IMAGE_PATH+'/original'+ '/' +images[0][1],bounding_boxes[i], new_height, new_width)
    new_box = calculate_resize[0],calculate_resize[1],calculate_resize[2],calculate_resize[3]
    new_area = calculate_resize[4]
    annotation['bbox'] = new_box
    annotation['area'] = new_area

#Change height/width
for image in data['images']:
    image['height']=new_height
    image['width']=new_width

with open(OUTPUTPATH_ANNOTATIONS + JSON_NAME, 'w') as destination_file:
    json.dump(data, destination_file, indent=2)    

Create Dataset

In [10]:
import json

images = data['images']
annotations = data['annotations']
categories = data['categories']

dataset = []

for annotation in annotations:
    image_info = next(image for image in images if image['id'] == annotation['image_id'])
    image_path = OUTPUT_PATH_IMAGES + '/'+ image_info['file_name']
    bbox = annotation['bbox']  # [x, y, width, height]
    class_id = annotation['category_id']

    xmin, ymin, width, height = bbox
    xmax, ymax = xmin + width, ymin + height

    dataset.append({
        'image_path': image_path,
        'bbox': [xmin, ymin, xmax, ymax],
        'class_id': class_id
    })

print('Annotation',len(dataset))
da =dataset[0]
print(dataset[0])
print(da['image_path'])


Annotation 697
{'image_path': 'workspace/images/224x224/000000000001.jpeg', 'bbox': [11.48, 60.11, 151.89, 110.05], 'class_id': 1}
workspace/images/224x224/000000000001.jpeg


In [11]:
import os

def get_file_list(directory):
    file_list = []
    for filename in os.listdir(directory):
        full_path = os.path.join(directory, filename)
        normalized_path = os.path.normpath(full_path)
        normalized_path = normalized_path.replace(os.path.sep, '/')
        if os.path.isfile(normalized_path):
            file_list.append(normalized_path)
    return file_list

In [12]:

def convert_annotations_to_array(coco_data, num_classes=41):
    annotations = coco_data['annotations']
    num_annotations = len(annotations)

    # Initialize empty arrays for bounding boxes and labels
    bounding_boxes = np.zeros((num_annotations, 4), dtype=np.float32)
    labels = np.zeros((num_annotations,), dtype=np.int32)

    for i, annotation in enumerate(annotations):
        # Extract bounding box coordinates
        bbox = annotation['bbox']
        bounding_boxes[i] = np.array([bbox[0], bbox[1], bbox[2], bbox[3]], dtype=np.float32)

        # Extract class label
        labels[i] = annotation['category_id']

    return bounding_boxes, labels


bounding_boxes, labels = convert_annotations_to_array(data)

#bbox_data = np.random.random((41, bounding_boxes))

trainTargets = {
    "class_label": labels,
    "bounding_box": bounding_boxes
}
print(bounding_boxes.shape)
"""
testTargets = {
    "class_label": testLabels,
    "bounding_box": testBBoxes
}
"""

(697, 4)


'\ntestTargets = {\n    "class_label": testLabels,\n    "bounding_box": testBBoxes\n}\n'

In [13]:
directory_path = OUTPUT_PATH_IMAGES
files = get_file_list(directory_path)


bbox_list=[]
class_id_list=[]
image_path_list=[]

for k in files:
    ann_bbox=[]
    ann_class_id=[]
    for i in dataset:
        d = i
        if d['image_path'] == k:
            ann_bbox.append(d['bbox'])
            #ann_bbox.append(d['class_id'])
            ann_class_id.append(d['class_id'])
    
    image = cv2.imread(d['image_path'])
    image = image / 255.0 

        
    bbox_list.append(ann_bbox)
    class_id_list.append(ann_class_id)
    image_path_list.append(image)

images_data = np.array(image_path_list)
bounding_boxes = np.array(bbox_list)
class_labels = np.array(class_id_list)

print(images_data.shape)
print(bounding_boxes.shape)
print(class_labels.shape)



(17, 224, 224, 3)
(17, 41, 4)
(17, 41)


In [14]:
class_ids = []

for cat in data['categories']:
    class_name = cat['name']
    class_ids.append(class_name)
class_mapping = dict(zip(range(len(class_ids)), class_ids))
print(len(class_ids))

42


In [15]:
import keras_cv
from keras_cv import bounding_box
from keras_cv import visualization

Using TensorFlow backend


Yolo

In [16]:
SPLIT_RATIO = 0.2
BATCH_SIZE = 4
LEARNING_RATE = 0.001
EPOCH = 5
GLOBAL_CLIPNORM = 10.0


In [17]:
backbone = keras_cv.models.YOLOV8Backbone.from_preset(
    "yolo_v8_s_backbone_coco"  # We will use yolov8 small backbone with coco weights
)
#Freeze base model
#base_model.trainable = False

In [18]:
yolo = keras_cv.models.YOLOV8Detector(
    num_classes=len(class_mapping),
    bounding_box_format="xywh",
    backbone=backbone,
    fpn_depth=1,
)


In [19]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=LEARNING_RATE,
    global_clipnorm=GLOBAL_CLIPNORM,
)

yolo.compile(optimizer=optimizer, classification_loss="binary_crossentropy", box_loss="ciou")
#yolo.compile(optimizer='adam', loss={'classes': 'categorical_crossentropy', 'boxes': 'mse'})
yolo.summary()

Model: "yolov8_detector"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 model (Functional)          {'P3': (None, None, None,    5089760   ['input_2[0][0]']             
                             128),                                                                
                              'P4': (None, None, None,                                            
                             256),                                                                
                              'P5': (None, None, None,                                            
                             512)}                                                  

                                                                                                  
 tf.concat_6 (TFOpLambda)    (None, None, None, 384)      0         ['tf.split_4[0][0]',          
                                                                     'tf.split_4[0][1]',          
                                                                     'pa_fpn_p4p5_pre_0_2[0][0]'] 
                                                                                                  
 pa_fpn_p4p5_output_conv (C  (None, None, None, 256)      98304     ['tf.concat_6[0][0]']         
 onv2D)                                                                                           
                                                                                                  
 pa_fpn_p4p5_output_bn (Bat  (None, None, None, 256)      1024      ['pa_fpn_p4p5_output_conv[0][0
 chNormalization)                                                   ]']                           
          

In [31]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss',  
                               patience=3,          
                               restore_best_weights=True)  

callbacks = [early_stopping]

yolo.fit(
    images_data, 
    {"classes": class_labels,"boxes": bounding_boxes},
    epochs=60,
    callbacks=early_stopping,
    #callbacks=[EvaluateCOCOMetricsCallback(val_ds, "model.h5")],
)

Epoch 1/60
1/1 [==============================] - 1s 853ms/step - loss: 0.5971 - box_loss: 0.2610 - class_loss: 0.3360
Epoch 2/60
1/1 [==============================] - 1s 798ms/step - loss: 0.5993 - box_loss: 0.2678 - class_loss: 0.3315
Epoch 3/60
1/1 [==============================] - 1s 832ms/step - loss: 0.5976 - box_loss: 0.2710 - class_loss: 0.3266
Epoch 4/60
1/1 [==============================] - 1s 814ms/step - loss: 0.5867 - box_loss: 0.2686 - class_loss: 0.3182
Epoch 5/60
1/1 [==============================] - 1s 827ms/step - loss: 0.5781 - box_loss: 0.2676 - class_loss: 0.3105
Epoch 6/60
1/1 [==============================] - 1s 815ms/step - loss: 0.5769 - box_loss: 0.2718 - class_loss: 0.3051
Epoch 7/60
1/1 [==============================] - 1s 839ms/step - loss: 0.5651 - box_loss: 0.2655 - class_loss: 0.2996
Epoch 8/60
1/1 [==============================] - 1s 848ms/step - loss: 0.5657 - box_loss: 0.2723 - class_loss: 0.2934
Epoch 9/60
1/1 [==============================] 

In [32]:
from keras.preprocessing import image as keras_image

img = OUTPUT_PATH_IMAGES + '/000000000001.jpeg'
img = keras_image.load_img(img, target_size=(new_height, new_width))
img_array = keras_image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0

print(yolo.predict(img_array))
boxes,confidence,classes,num_detections=yolo.predict(img_array)



1/1 [==============================] - 0s 57ms/step
{'boxes': array([[[  96.93915  ,  132.06253  ,  190.81451  ,  207.33511  ],
        [  48.796135 ,  135.99123  ,   89.20704  ,  138.59422  ],
        [   9.963955 ,  131.27667  ,   78.30206  ,  138.25867  ],
        [  33.774723 ,  137.17574  ,   87.57179  ,  138.06252  ],
        [ 119.56271  ,  145.9644   ,  131.02914  ,  136.30507  ],
        [ -10.205013 ,  135.30757  ,   77.01924  ,  143.17174  ],
        [  57.594707 ,  144.79167  ,   99.12862  ,  137.33174  ],
        [  19.857788 ,  137.6918   ,   84.773315 ,  138.00125  ],
        [ 102.42645  ,  148.27646  ,  123.04248  ,  135.04178  ],
        [ -49.36269  ,  139.9113   ,  109.52658  ,  135.01173  ],
        [ -20.69986  ,  129.18628  ,  190.70314  ,  210.93924  ],
        [ -10.557405 ,  111.20355  ,   80.35826  ,  140.11537  ],
        [  73.28781  ,  136.73502  ,   97.47667  ,  137.04828  ],
        [ -51.859535 ,  117.39769  ,  113.76164  ,  133.66429  ],
        [ -14.

New Top Layer

In [22]:
base_model = keras_cv.models.MobileNetV3Backbone.from_preset(
    "mobilenet_v3_large_imagenet",
    load_weights=False,
)
#Freeze base model
base_model.trainable = False

In [23]:
import keras

inputs = keras.Input(shape=(new_height, new_width, 3))
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)

class_id_output = keras.layers.Dense(41, activation='softmax', name='class_id')(x)

bbox_output = keras.layers.Dense(4, name='bbox')(x)
model = keras.Model(inputs, [class_id_output, bbox_output]) 

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss={'class_id': 'sparse_categorical_crossentropy', 'bbox': 'mean_squared_error'},#Regression für bbox
    metrics={'class_id': 'accuracy'}
)
print(images_data.shape)
print(bounding_boxes.shape)
print(class_labels.shape)

model.fit(images_data,  {"class_id": class_labels,"bbox": bounding_boxes}, epochs=20, validation_split=0.2)

model.summary()

(17, 224, 224, 3)
(17, 41, 4)
(17, 41)
Epoch 1/20


InvalidArgumentError: Graph execution error:

Detected at node gradient_tape/mean_squared_error/BroadcastGradientArgs defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\Tristan\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\Tristan\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1046, in launch_instance

  File "C:\Users\Tristan\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 736, in start

  File "C:\Users\Tristan\AppData\Roaming\Python\Python311\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "c:\Users\Tristan\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 607, in run_forever

  File "c:\Users\Tristan\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 1922, in _run_once

  File

In [33]:
arr= [0.44037277,  0.40316486,  0.38636532,  0.37286654,  0.36384788,
         0.3578291 ,  0.3546168 ,  0.34756556,  0.31302404,  0.2949059 ,
         0.2924991 ,  0.29208964,  0.28262794,  0.27496898,  0.26644573,
         0.2644392 ,  0.26355702,  0.2572701 ,  0.25449604,  0.25369182,
         0.25073034,  0.2492903 ,  0.24091788,  0.24013893,  0.23905557,
         0.23537159,  0.23481429,  0.23091015,  0.23083682,  0.22919336,
         0.22898686,  0.22520238,  0.2242464 ,  0.21672519,  0.21456368,
         0.21431774,  0.21165143,  0.21079609,  0.2101818 ,  0.20937951,
         0.2066574 ,  0.20618404,  0.20534521,  0.2045097 ,  0.20254572]
print(len(arr))

45
